# Sim-SOENs
## A Simulator for Superconducting Optoelectronic Networks

### Part 1: A Simple Neuron

### Part 2: Dendritic Arbors

### Part 3: Input-Output

### Part 4: Computing with A Single Neuron

### Part 5: Networking and Dynamics